# Cálculo y análisis de la tasa de deforestación

In [8]:
#python3
import pandas as pd
import numpy as np
import os
import sys
sys.path.insert(0, os.path.abspath('../..'))
from landecology import deforestation_rate as dr

## Datos

In [3]:
deforestation_df = pd.read_csv("./example_data_set.csv", index_col=0)
deforestation_df

,area_bosque,area_pastos,area_nubes,area_sindatos
año,,,,
2000,16149333600,6367233600,433800,77598000
2010,14814712800,7660449000,41839200,77598000
2020,13526236800,8754103800,236660400,77598000


## Resultados

### Tasa de cambio anual del bosque (Puyravaud)

In [4]:
# Define analysis parameters

coberture_keyname = "area_bosque"
years = sorted([2010, 2000])
parameters: dict[str, float] = {
            "area_t1": deforestation_df.loc[years[0]][coberture_keyname],
            "area_t2": deforestation_df.loc[years[1]][coberture_keyname],
            "year_t1": years[0],
            "year_t2": years[1],
        }
rate_type = dr.DeforestationFormula.FOREST_CHANGE_PUYRAVAUD

In [29]:
# Perform analysis
forest_change = dr.DeforestationCalculator.calculate_deforestation_rate(rate_type, **parameters)
print("La tasa de cambio de bosque (basado en {}) es: {:.2f}%".format(rate_type.value, forest_change*100))

La tasa de cambio de bosque (basado en Puyravaud) es: -0.86%


### Proyección para el 2020

La tasa de cambio propuesta por Puyravaud (2003) es equivalente a lo que en ecología de poblaciones se denomina "tasa intrínseca de crecimiento r".

Bajo el modelo exponencial:

$A_t = A_0 \cdot e^{r(t-t_0)}$

In [36]:
actual_area_2020 = deforestation_df.loc[2020][coberture_keyname]
inferred_area_2020 = (lambda t: parameters["area_t1"] * np.power(np.e, forest_change * t))(2020 - parameters["year_t1"])
print("Area medida: {}\nArea proyectada: {:.0f}".format(actual_area_2020, inferred_area_2020))
absolute_error = inferred_area_2020 - actual_area_2020
percent_error = np.abs(absolute_error/actual_area_2020)*100
print("Error absoluto: {:0f}\nError porcentual: {:.2f}".format(absolute_error,percent_error))

Area medida: 13526236800
Area proyectada: 13590388358
Error absoluto: 64151557.974340
Error porcentual: 0.47


## Referencias

Puyravaud, J.-P. (2003). Standardizing the calculation of the annual rate of deforestation. Forest Ecology and Management, 177(1–3), 593–596. https://doi.org/10.1016/S0378-1127(02)00335-3